In [1]:
import requests
import json
import pandas as pd
import numpy as np
from datetime import datetime
from tqdm import tqdm_notebook
import bs4
import re
import os
from requests.exceptions import ConnectionError
from urllib3.exceptions import ProtocolError
from http.client import RemoteDisconnected
import time

import warnings
warnings.filterwarnings("ignore")

In [14]:
%load_ext autoreload
%autoreload 2

from sofa_score_scrap import *

In [15]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36"
}

#### Dê/Para de Siglas

In [158]:
de_para_siglas = pd.read_excel("de_para_siglas_epl.xlsx")
de_para_siglas["time"] = de_para_siglas["time"].astype(str)
de_para_siglas["time"] = de_para_siglas["time"].apply(str.strip)
de_para_siglas.set_index("time", inplace=True)

### Get Incidentes

In [4]:
resp = requests.get('https://api.sofascore.com/api/v1/event/8684164/incidents', headers=headers)

In [6]:
resp.json()

{'incidents': [{'text': 'FT',
   'homeScore': 2,
   'awayScore': 1,
   'isLive': False,
   'time': 90,
   'addedTime': 999,
   'incidentType': 'period'},
  {'playerIn': {'name': 'Billy Gilmour',
    'slug': 'gilmour-billy',
    'shortName': 'B. Gilmour',
    'position': 'M',
    'userCount': 2700,
    'id': 907668},
   'playerOut': {'name': 'Christian Pulišić',
    'slug': 'christian-pulisic',
    'shortName': 'C. Pulišić',
    'position': 'M',
    'userCount': 20255,
    'id': 817957},
   'id': 119665608,
   'time': 90,
   'addedTime': 1,
   'isHome': True,
   'incidentType': 'substitution'},
  {'playerIn': {'name': 'Pedro',
    'slug': 'pedro',
    'shortName': 'Pedro',
    'position': 'F',
    'userCount': 5316,
    'id': 35428},
   'playerOut': {'name': 'Mason Mount',
    'slug': 'mason-mount',
    'shortName': 'M. Mount',
    'position': 'M',
    'userCount': 11328,
    'id': 836694},
   'id': 119665607,
   'time': 90,
   'addedTime': 1,
   'isHome': True,
   'incidentType': 'subs

In [187]:
incidents_per_type = {}

for el in resp_incidents.json()["incidents"]:
    if not el['incidentType'] in incidents_per_type:
        incidents_per_type[el['incidentType']] = []
    incidents_per_type[el['incidentType']].append(el)

In [189]:
incidents_per_type.keys()

dict_keys(['period', 'substitution', 'injuryTime', 'goal', 'card', 'varDecision'])

In [17]:
for key in incidents_per_type:
    incidents_per_type[key] = pd.DataFrame(incidents_per_type[key])

In [193]:
incidents_dict = get_incidents_database(resp_incidents)

In [196]:
incidents_dict['goal']

,homeScore,awayScore,player,id,time,isHome,incidentClass,incidentType
0,2,1,"{'name': 'Willian', 'slug': 'willian', 'shortN...",130804277,78,True,penalty,goal
1,1,1,"{'name': 'Kevin De Bruyne', 'slug': 'kevin-de-...",130804278,55,False,regular,goal
2,1,0,"{'name': 'Christian Pulišić', 'slug': 'christi...",130804276,36,True,regular,goal


In [8]:
df_incidents = pd.DataFrame(resp.json()["incidents"])

In [11]:
df_incidents[df_incidents.incidentType == 'goal']

,text,homeScore,awayScore,isLive,time,addedTime,incidentType,playerIn,playerOut,id,isHome,length,player,incidentClass,playerName,reason,confirmed
4,NaN,2.0,1.0,NaN,78,NaN,goal,NaN,NaN,130804277.0,True,NaN,"{'name': 'Willian', 'slug': 'willian', 'shortN...",penalty,NaN,NaN,NaN
13,NaN,1.0,1.0,NaN,55,NaN,goal,NaN,NaN,130804278.0,False,NaN,"{'name': 'Kevin De Bruyne', 'slug': 'kevin-de-...",regular,NaN,NaN,NaN
19,NaN,1.0,0.0,NaN,36,NaN,goal,NaN,NaN,130804276.0,True,NaN,"{'name': 'Christian Pulišić', 'slug': 'christi...",regular,NaN,NaN,NaN


In [10]:
df_incidents.incidentType.value_counts()

substitution    8
card            3
goal            3
period          2
varDecision     2
injuryTime      2
Name: incidentType, dtype: int64

### Players Stats 

In [29]:
# s.get('https://www.sofascore.cddom/pt/atletico-mineiro-internacional/qOsCO', headers={'x-test2': 'true'})
resp = requests.get("https://www.sofascore.com/event/7724681/statistics/players/json?_=154328071", headers=headers)

In [30]:
players_df = pd.DataFrame(resp.json()["players"])

In [35]:
get_per_player_data(players_df, de_para_siglas)

,name,slug,shortName,id,disabled,national,team_name,team_id,goals,goalAssist,...,totalLongBalls,possessionLost,fouls,wasFouled,offsides,saves,punches,runsOut,goodHighClaim,team
game,,,,,,,,,,,,,,,,,,,,,
INT X CAM 2018-11-21,Juan Cazares,atletico-mineiro,J. Cazares,158411,False,False,Atlético Mineiro,1977,1,1,...,2 (2),17,2,1,NaN,NaN,NaN,NaN,NaN,CAM
INT X CAM 2018-11-21,Andres D'Alessandro,internacional,A. D'Alessandro,17639,NaN,False,Internacional,1966,1,,...,5 (2),23,1,3,NaN,NaN,NaN,NaN,NaN,INT
INT X CAM 2018-11-21,Víctor Cuesta,internacional,V. Cuesta,587148,NaN,False,Internacional,1966,,,...,11 (7),13,2,3,NaN,NaN,NaN,NaN,NaN,INT
INT X CAM 2018-11-21,David Terans,atletico-mineiro,D. Terans,586662,False,False,Atlético Mineiro,1977,1,,...,2 (1),5,1,0,NaN,NaN,NaN,NaN,NaN,CAM
INT X CAM 2018-11-21,Nico López,internacional,N. López,255881,NaN,False,Internacional,1966,,,...,7 (4),15,1,1,NaN,NaN,NaN,NaN,NaN,INT
INT X CAM 2018-11-21,Patric,atletico-mineiro,Patric,78853,False,False,Atlético Mineiro,1977,,,...,8 (1),22,1,1,NaN,NaN,NaN,NaN,NaN,CAM
INT X CAM 2018-11-21,Luan,atletico-mineiro,Luan,243427,False,False,Atlético Mineiro,1977,,,...,1 (0),8,4,1,NaN,NaN,NaN,NaN,NaN,CAM
INT X CAM 2018-11-21,Emerson Santos,internacional,E. Santos,844015,NaN,False,Internacional,1966,,,...,4 (2),9,0,0,NaN,NaN,NaN,NaN,NaN,INT
INT X CAM 2018-11-21,Fabiano,internacional,F. Leismann,551304,NaN,False,Internacional,1966,,,...,5 (2),20,2,0,NaN,NaN,NaN,NaN,NaN,INT


### Game Stats

In [43]:
resp = requests.get("https://www.sofascore.com/event/7724681/general/json", headers=headers)

In [38]:
pd.DataFrame(game_statistics(resp, players_df, de_para_siglas, period=0), index=[0]).set_index("game")

,team_away,team_home,data,hora,away_score,home_score,result,Ball possession_home,Ball possession_away,Total shots_home,...,form_minute_82,form_minute_83,form_minute_84,form_minute_85,form_minute_86,form_minute_87,form_minute_88,form_minute_89,form_minute_90,form_minute_90.5
game,,,,,,,,,,,,,,,,,,,,,
INT X CAM 2018-11-21,CAM,INT,2018-11-21,19:30:00,2,1,-1,54%,46%,15,...,22,73,41,24,1,1,63,29,28,-49


In [48]:
resp.json()["statistics"]["periods"][0]['period']

'ALL'

### Pega links dos jogos históricos

#### Pega os IDs das temporadas

In [95]:
# comp = requests.get("https://www.sofascore.com/pt/torneio/futebol/brazil/brasileiro-serie-a/325", headers=headers)
# comp = requests.get("https://www.sofascore.com/pt/torneio/futebol/england/premier-league/17", headers=headers)
# comp = requests.get("https://www.sofascore.com/pt/torneio/futebol/spain/laliga/8", headers=headers)
# comp = requests.get("https://www.sofascore.com/pt/torneio/futebol/germany/bundesliga/35", headers=headers)
# comp = requests.get("https://www.sofascore.com/pt/torneio/futebol/italy/serie-a/23", headers=headers)
comp = requests.get("https://www.sofascore.com/pt/torneio/futebol/germany/2-bundesliga/44", headers=headers)
c = bs4.BeautifulSoup(comp.content)

In [96]:
temporadas = c.find_all("script", attrs={"id":"__NEXT_DATA__"})

In [97]:
temp = str(temporadas[0]).replace('<script id="__NEXT_DATA__" type="application/json">', '')
temp = temp.replace('</script>', '')

In [98]:
temp_j = json.loads(temp)

In [99]:
season_ids = temp_j['props']['initialProps']['pageProps']['seasons']

In [100]:
season_ids

[{'name': '2nd Bundesliga 19/20', 'year': '19/20', 'id': 24043},
 {'name': '2nd Bundesliga 18/19', 'year': '18/19', 'id': 17598},
 {'name': '2nd Bundesliga 17/18', 'year': '17/18', 'id': 13476},
 {'name': '2nd Bundesliga 16/17', 'year': '16/17', 'id': 11819},
 {'name': '2nd Bundesliga 15/16', 'year': '15/16', 'id': 10420},
 {'name': '2nd Bundesliga 14/15', 'year': '14/15', 'id': 8240},
 {'name': '2nd Bundesliga 13/14', 'year': '13/14', 'id': 6335},
 {'name': '2nd Bundesliga 12/13', 'year': '12/13', 'id': 4794},
 {'name': '2nd Bundesliga 11/12', 'year': '11/12', 'id': 3406},
 {'name': '2nd Bundesliga 10/11', 'year': '10/11', 'id': 2812},
 {'name': '2. Bundesliga 09/10', 'year': '09/10', 'id': 2192},
 {'name': '2. Bundesliga 08/09', 'year': '08/09', 'id': 1567}]

In [101]:
json.dump(season_ids, open("IDs_2nd_Bundesliga_seasons.json", 'w'))

### Temporada

In [102]:
temporadas = json.load(open("IDs_2nd_Bundesliga_seasons.json", 'r'))

In [103]:
# temporadas = c.find_all("a", attrs={"class":"pointer js-uniqueTournament-page-load-season"})

id_temporada = {a['year']: a['id'] for a in temporadas}
id_temporada

{'19/20': 24043,
 '18/19': 17598,
 '17/18': 13476,
 '16/17': 11819,
 '15/16': 10420,
 '14/15': 8240,
 '13/14': 6335,
 '12/13': 4794,
 '11/12': 3406,
 '10/11': 2812,
 '09/10': 2192,
 '08/09': 1567}

In [104]:
r = resp.json()

In [105]:
r = bs4.BeautifulSoup(resp.text)

## Pega Links dos jogos por Rodadas

#### Página Inicial

In [106]:
# comp = requests.get("https://www.sofascore.com/pt/torneio/futebol/brazil/brasileiro-serie-a/325", headers=headers)
# comp = requests.get("https://www.sofascore.com/pt/torneio/futebol/england/premier-league/17", headers=headers)
# comp = requests.get("https://www.sofascore.com/pt/torneio/futebol/spain/laliga/8", headers=headers)
# comp = requests.get("https://www.sofascore.com/pt/torneio/futebol/germany/bundesliga/35", headers=headers)
# comp = requests.get("https://www.sofascore.com/pt/torneio/futebol/italy/serie-a/23", headers=headers)
comp = requests.get("https://www.sofascore.com/pt/torneio/futebol/germany/2-bundesliga/44", headers=headers)

In [107]:
s = requests.Session()
s.get(comp.url, headers=headers)

<Response [200]>

#### Para cada temporada

In [122]:
de_para_siglas = pd.read_excel("de_para_siglas_bundesliga.xlsx")
de_para_siglas["time"] = de_para_siglas["time"].astype(str)
de_para_siglas["time"] = de_para_siglas["time"].apply(str.strip)
de_para_siglas.set_index("time", inplace=True)

for temp in list(id_temporada.keys()):
    ret = []
    
    for rodada in tqdm_notebook(range(1, 35)):
        # print(temp[0] + ": " + str(rodada), end="\r")
        resp = s.get("https://www.sofascore.com/u-tournament/44/season/"+ str(id_temporada[temp]) + "/matches/round/" + str(rodada), headers=headers)
        resp.encoding = 'UTF-8'
        ret.append(get_info_rodada(resp, de_para_siglas))
    
    ret = pd.concat(ret)
    ret.to_csv("links_sofa_score/sofa_score_links_2nd_GER_"+temp.replace("/", "_")+".csv", sep=";")

KeyError: 'Rot Weiss Ahlen'

### Pega informações dos jogos históricos

In [2]:
base = pd.read_csv("links_sofa_score/sofa_score_links_EPL_18_19.csv", sep=";")

In [124]:
list(reversed(sorted([x for x in os.listdir("links_sofa_score/") if '2nd_GER' in x])))

['sofa_score_links_2nd_GER_19_20.csv',
 'sofa_score_links_2nd_GER_18_19.csv',
 'sofa_score_links_2nd_GER_17_18.csv',
 'sofa_score_links_2nd_GER_16_17.csv',
 'sofa_score_links_2nd_GER_15_16.csv',
 'sofa_score_links_2nd_GER_14_15.csv',
 'sofa_score_links_2nd_GER_13_14.csv',
 'sofa_score_links_2nd_GER_12_13.csv',
 'sofa_score_links_2nd_GER_11_12.csv',
 'sofa_score_links_2nd_GER_10_11.csv']

In [7]:
API_HEADERS = {
    'accept': '*/*',
    'accept-language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
    'cache-control': 'max-age=0',
    'if-none-match': 'W/"8b4895fbff"',
    'origin': 'https://www.sofascore.com',
    # 'referer': 'https://www.sofascore.com/pt/chelsea-manchester-city/rN'
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.106 Safari/537.36'
}

In [125]:
prefix = '2nd_GER'
lista = list(reversed(sorted([x for x in os.listdir("links_sofa_score/") if prefix in x])))

In [126]:
s = requests.Session()
s.get("https://www.sofascore.com/pt/torneio/futebol/brazil/brasileiro-serie-a/325", headers=headers)

<Response [200]>

In [ ]:
de_para_siglas = pd.read_excel("de_para_siglas_bundesliga.xlsx")
de_para_siglas["time"] = de_para_siglas["time"].astype(str)
de_para_siglas["time"] = de_para_siglas["time"].apply(str.strip)
de_para_siglas.set_index("time", inplace=True)

erros = []

for file in lista[2:]:
    base = pd.read_csv("links_sofa_score/" + file, sep=";")
    
    base[(base.home_score != -1) & (base.away_score != -1)]
    
    resp_game = []
    resp_players = []
    resp_incidents_df = {}
    for key in ['period', 'substitution', 'injuryTime', 'goal', 'card', 'varDecision']:
        resp_incidents_df[key] = []
        
    game_id = 0
    
    # name = file[-11:-4]
    # name = file[-13:-4]
    name = file[-17:-4]
    
    tentativas = 3
    while(tentativas > 0):
        try:
            while (game_id < len(base.id)):
                print(name + " Game #" + str(game_id), end="\r")
                
                API_HEADERS['referer'] = base.link.iloc[game_id]
                
                # # # # # # # # #
                # DADOS DO JOGO #
                # # # # # # # # #
                resp = s.get("https://www.sofascore.com/event/" + str(base.id.iloc[game_id]) + "/general/json",
                             headers=API_HEADERS, timeout=10)

                # # # # # # # # # # #
                # DADOS POR JOGADOR #
                # # # # # # # # # # #
                resp2 = s.get("https://www.sofascore.com/event/" + str(base.id.iloc[game_id]) + "/statistics/players/json",
                              headers=API_HEADERS, timeout=10)
                resp2.encoding = 'UTF-8'
                
                # # # # # # #
                # INCIDENTS #
                # # # # # # #
                resp_incidents = s.get('https://api.sofascore.com/api/v1/event/'+ str(base.id.iloc[game_id]) +'/incidents',
                                      headers=API_HEADERS, timeout=10)
                resp_incidents.encoding = 'UTF-8'
                
                
                # time.sleep(np.random.choice([2, 3, 4, 5]))
                time.sleep(np.random.choice([0.01, 0.1, 0.5]))
                
                if "Statistics not found." in str(resp2.content):
                    erros.append(base.id.iloc[game_id])
                    tentativas = 0
                    game_id += 1
                    continue
                    
                try:
                    players_df = pd.DataFrame(resp2.json()["players"])  
                except KeyError:
                    players_df = None
                    pass
        
                ### Guarda estatisticas do jogo
                game_df = pd.DataFrame(game_statistics(resp, players_df, de_para_siglas), index=[0])
                game_df['game'] = base.game.iloc[game_id]
                resp_game.append(
                    game_df.set_index("game")
                )
                
                ### Guarda statisticas dos jogadores
                if players_df is not None and not players_df.empty:
                    per_player_data_df = get_per_player_data(players_df, de_para_siglas)
                    per_player_data_df['game'] = base.game.iloc[game_id]
                    resp_players.append(per_player_data_df.set_index("game")) 
                
                
                ### Guarda Incidents 
                incidents_df = get_incidents_database(resp_incidents)
                for key in incidents_df:
                    # Adiciona coluna "GAME" para index
                    incidents_df[key]['game'] = base.game.iloc[game_id]
                    incidents_df[key].set_index('game', inplace=True)
                    
                    resp_incidents_df[key].append(
                        incidents_df[key].copy()
                    )
                
                tentativas = 0
                game_id += 1
        except (ConnectionError, ProtocolError, RemoteDisconnected):
            print("Error")
            time.sleep(2)
            tentativas -= 1

    print("Guardando dados capturados...")
    
    try:
        os.makedirs(prefix + "_data_sofa_score/" + name + "/")
    except:
        pass
    
    ret_game = pd.concat(resp_game)
    ret_game.drop_duplicates().to_csv(prefix + "_data_sofa_score/" + name + "/game_data_"+ name +".csv", sep=";")

    ret_players = pd.concat(resp_players)
    ret_players.drop_duplicates().to_csv(prefix + "_data_sofa_score/" + name + "/players_data_"+ name +".csv", sep=";")

    for key in resp_incidents_df:
        if len(resp_incidents_df[key]) > 0:
            resp_incidents_concat = pd.concat(resp_incidents_df[key])
            try:
                resp_incidents_concat = resp_incidents_concat.drop_duplicates()
            except: 
                pass
            resp_incidents_concat.to_csv(prefix + "_data_sofa_score/" + name + "/incidents_" + key + "_data_" + name + ".csv", sep=';')
    
    print("Com sucesso!")
    print()
    
    resp_game = []
    resp_players = []
    resp_incidents_df = {}
    for key in ['period', 'substitution', 'injuryTime', 'goal', 'card', 'varDecision']:
        resp_incidents_df[key] = []
        
    game_id = 0

Guardando dados capturados...
Com sucesso!

